In [10]:
import csv
import pandas as pd
narratives = ['benefits/connections/afghanistan', 'benefits/covid', 'benefits/development/energy', 'benefits/development/maritime', 'benefits/development/roads', 'benefits/jobs', 'controversies/china/border', 'controversies/china/debt', 'controversies/china/exploitation', 'controversies/china/funding', 'controversies/china/naval', 'controversies/china/uighur', 'controversies/pakistan/army', 'controversies/pakistan/bajwa', 'controversies/pakistan/baloch', 'controversies/pakistan/students', 'leadership/bajwa', 'leadership/khan', 'leadership/sharif', 'opposition/kashmir', 'opposition/propaganda']
selected_nar = narratives
url2lot = dict()
url2title = dict()

with open('lotclass_predictions_append.csv') as fin:
    reader = csv.DictReader(fin)
    for js in reader:
        u = js['url']
        url2lot[u] = {i: (float(js[i]) if i in js else 0.0) for i in narratives}
        

import json
from collections import defaultdict
    
chosen_url = set()
for nar in selected_nar:
    urls = list(url2lot)
    urls = sorted(urls, key=lambda u:url2lot[u][nar], reverse=True)
    for u in urls[:300]:
        chosen_url.add(u)

import json
keywords = json.load(open('keywords.txt'))
global_cnt = {"benefits/covid": 485344, "benefits/development/roads": 74575, "benefits/development/maritime": 44606, "benefits/development/energy": 55211, "leadership/bajwa": 18469, "opposition/propaganda": 179989, "opposition/kashmir": 484026, "benefits/jobs": 34899, "leadership/khan": 164996, "controversies/china/uighur": 49202, "controversies/china/border": 1280130, "controversies/china/debt": 55764, "controversies/china/exploitation": 118370, "controversies/pakistan/students": 10867, "controversies/pakistan/baloch": 172937, "controversies/pakistan/army": 82715, "benefits/connections/afghanistan": 5863, "leadership/sharif": 92467, "controversies/pakistan/bajwa": 15371, "controversies/china/funding": 1425, "controversies/china/naval": 256}

def string_match(text):
    nar_cnt = defaultdict(int)
    found = 0
    for nar in narratives:
        for key in keywords[nar]:
            if text.lower().find(key) != -1:
                nar_cnt[nar] += 1
                found += 1
    if found >= 1:
        return {i: nar_cnt[i] for i in narratives}
    else:
        return None
    
for u in url2lot:
    string_match_result = string_match(url2title[u] if u in url2title else '')
    if u in chosen_url:
        for nar in narratives:
            if nar not in url2lot[u]:
                url2lot[u][nar] = 0.0
        if len(url2lot[u]) != 21:
            print(len(url2lot[u]), url2lot[u])
            assert False
    else:
        url2lot[u] = string_match_result
#         assert len(url2west[u]) == 21
        
with open('url2lot_prob_append.json', 'w') as fout:
    for u in url2lot:
        i = url2lot[u]
        if i:
            fout.write(json.dumps({'url': u, 'prob': i}) + '\n')
        